# **Graph plan**
Graph plan is a **very efficient**, **correct** and **complete** algorithm (can be **optimal** too). Returns either the shortest possible plan or returns an inconsistency.

It inserts the concept of *time* with **timepoints**.
This planner builds a datastructure called **planning graphs** and at each step this structure is extended.
They are offline, generative and relies under close world assumption.

Graph plan inherits:
* **Early commitment** from linear planner;
* **Partially ordered action** (generates parralel plans) from non-linear planner.

Actions are represented as the ones in STRIPS:
* Preconditions;
* Add-list;
* Delete-list.

There is a dummy action called **no-op** that doesn't change the state to deal with the frame problem.
States are represented as a sets of predicates that are true in a given state.
Objects have a type!

The planning graph is a **directed leveled graph** where:
* Nodes belong to different levels;
* Arcs connect nodes in adjacent levels;
* Level 0 corresponds to the initial state.

In the planning graph **proposition levels** (nodes represent propositions) and **action levels** (nodes represent actions) are interleaved and correspond to increasing timepoints. Interfearing actions and proposition in a certain timestep can appear.

Arcs are divided into:
* Precondition arcs: `proposition` $\to$ `action`;
* Add-arcs: `action` $\to$ `proposition`;
* Delete-arcs: `action` $\to$ `proposition`.

In each timestep an action $A$ can be inserted if in the previous timestep all preconditions of $A$ exist.

Each action level contains:
* All actions that are applicable at that timestep;
* Constraints connecting pairs of actions that cannot be performed simultaneously.

Each proposition level contains all literal that might result from any choice of actions in the previous timestep, including the no-op.
Note that the construction process of the planning graph doesn't imply any choice on the selection of the action that will be inserted in the plan.

# Inconsistencies
During the construction of the planning graph, inconsistencies are identified: two actions or two propositions can be inconsistent in the same timestep.
In that case they're **mutually exclusive**: they can't appear together in a plan but they may appear together in the same level of the planning graph.

**Inconsistent effects**: one action negates the effect of another;

**Interference**: an action deletes a precondition of the other;

**Competing needs**: two actions that have mutually exclusive preconditons.

Two proposition are inconsistent if:
* One is the negation of the other;
* If all the ways to reach them are mutually exclusive.
* There can be domain dependent inconsistencies.

# The algorithm

**Initialization**
All true propositions in the initial states are inserted in the first *proposition level*.

**Creation of the action level**
* For every operator and for every way to unify its preconditions to propositions in the previous proposition level, enter an action node if two propositions aren't labelled as **mutually exclusive**;
* In addition, for every proposition in the previous proposition level, add a no-op operator;
* Check if the action nodes don't interfere to each other, otherwise mark them as mutually exclusive.

**Creation of the proposition level**
* For each action node in the previous level, add propositions in its **add-list** through **solid arcs** and add *dotted arcs* connected to the preopositions in the *delete-list*;
* Do the same process for the no-op operators;
* Mark as a mutually exclusive two propositions such that all the ways to achieve the first are incompatible with all the ways to reach the second.


# Extraction of a valid plan
Once the planning graph is built we have to extract a **valid plan**: a connected and consistent subgraph of the planning graph.

A plan is valid if:
* Actions in the same timestep can be perfomed in any order without interference;
* Propositions at the same timestep aren't mutually exclusive;
* The last timestep contains all the literals of the goal and they aren't mutually exclusive.

The **inconsistencies** found by the algorithm **prune paths in the search tree**.

> function `GraphPlan(problem)`:
* `graph` $\leftarrow$ `InitialGraph(problem)`;
* `targets` $\leftarrow$ `Goal(problem)`;
* loop:
1. If objects aren't mutually exclusive in the last step then:
* `Sol` $\leftarrow$ `ExtractSolution(graph, objectives)`;
* If `Sol` $\ne$ `fail` then return `Sol`;
* Else, if `LevelOff(graph)` then return `fail`= `GraphExpand (graph, problem)`

The first node contains the initial planning graph that contains only one timestep with the true propositions in the initial state. The initial graph is extracted from `InitialGraph(problem)`.

The goal to reach is extracted from the function `Goal(problem)`.

If goals aren't mutually exclusive in the last level then the plan **could** include a valid plan. The valid plan is extracted through **backward search** from the function `ExtractSolution(graph, objectives)`, that provides a solution or a failure:
* It proceeds level by level to better exploit the mutual exclusion constraints;
* Recursive method: given a set of goals at time $t$ the algorithms looks for a set of actions at time $t-1$ who have such goals as add-effects. The actions shouldn't be mutually exclusive;
* The search is **hybrid breadth/depth first** and **complete**. 

**Memoization**: if at some step of the search a subset of goals is not satisfiable, graph plan saves this result in an hash table. Whenevere the same subset of goals is selected in the future will automatically fail.

# Fast forward (FF)
It's an extremely efficient heuristic planner where in each state $S$ an estiamtion of the distance from the goal is computed.

The basic operation is a merge of hill climbing and A*:
1. From a state $S$ examines all successors $S'$;
1. If a successor state $S^*$ better than $S$ exists then move on it and go back to point 1;
1. If there is no state with a better evaluation then perform a complete A* search using the same heuristic.

Given a problem $P$, a state $S$ and a goal $G$, FF considers a relaxed problem $P_+$ derived from $P$ neglecting delete-effects actions. FF solves $P_+$ with graph plan and then, the number of actions in the resulting plan it's used as heuristics.

FF uses a so-called **enforced hill climbing**, in practice it's a complete breadth first search. A solution is always found unless the current state is a dead end.